In [ ]:
!pip install facenet-pytorch

In [ ]:
from facenet_pytorch import MTCNN
import cv2
import os
import glob
import tqdm
import torch
import json
import glob
import numpy as np
import copy
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
import tqdm

In [ ]:
# Mount Google Drive to access files in Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ✅ Set the device to MPS(for Mac) if available, otherwise fallback to CUDA or CPU
device = torch.device("mps") if torch.backends.mps.is_available() else (
  torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)
print(f"Using device: {device}")

In [ ]:
# Function to calculate the average number of frames per video
def average_frame_count(input_file_path):
  input_path = f'{input_file_path}/*.mp4'
  video_files = glob.glob(input_path)
  frame_count = []
  video_list = []
  short_frame=[]
  short_frame_count=[]
  for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
      short_frame.append(video_file)
      short_frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
      #video_files.remove(video_file) # 삭제 대신 리스트에 추가하여 목록 관리
      continue
    video_list.append(video_file) # 프레임 150 이상인 영상들
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
    cap.release()

  print("Total number of videos: " , len(frame_count))
  print('Average frame per video:',np.mean(frame_count))
  print('Short frame video:',len(short_frame))

  return video_list



# Extract frame from video - 비디오 파일에서 프레임 추출
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image

# Create face videos by detecting faces in each frame
def create_face_videos(path_list, out_dir):
    already_present_count = glob.glob(out_dir + '/*.mp4')
    print("No of videos already present:", len(already_present_count))

    for path in tqdm.tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.basename(path))

        # Skip if the video already exists
        if os.path.exists(out_path):
            print("File already exists:", out_path)
            continue

        frames = []
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MJPG'), 30, (224,224))

        for idx, frame in enumerate(frame_extract(path)):
            # if idx > 150:
            #     break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Face detection using MTCNN
            box, _ = mtcnn.detect(rgb_frame)

            if box is not None:
                x1, y1, x2, y2 = map(int, box[0])  # Use the first detected face
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(frame.shape[1], x2)
                y2 = min(frame.shape[0], y2)

                face = frame[y1:y2, x1:x2]
                if face.size != 0:
                    resized_face = cv2.resize(face, (224,224))
                    out.write(resized_face)

        out.release()

In [ ]:

input_file_path='/content/drive/MyDrive/Capstone/Dataset/FaceForensics++_C23'
output_file_path='/content/drive/MyDrive/Capstone/Dataset/ff++'
data_details="NeuralTextures"

input_path = f"{input_file_path}/{data_details}"
output_path = f"{output_file_path}/{data_details}"

# Initialize the MTCNN face detector
mtcnn = MTCNN(keep_all=False, device=device)

video_files = average_frame_count(input_path)
create_face_videos(video_files, output_path)